<a href="https://colab.research.google.com/github/danielrodriguez007/Data_Analysis/blob/main/Flipkart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
plt.style.use('fivethirtyeight')

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/amankharwal/Website-data/master/flipkart_reviews.csv')
data.head()

,Product_name,Review,Rating
0,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Best under 60k Great performanceI got it for a...,5
1,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Good perfomence...,5
2,Lenovo Ideapad Gaming 3 Ryzen 5 Hexa Core 5600...,Great performance but usually it has also that...,5
3,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,My wife is so happy and best product 👌🏻😘,5
4,DELL Inspiron Athlon Dual Core 3050U - (4 GB/2...,"Light weight laptop with new amazing features,...",5


In [3]:
data.isnull().sum()

Product_name    0
Review          0
Rating          0
dtype: int64

In [4]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["Review"] = data["Review"].apply(clean)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import plotly.express as px 

ratings = data['Rating'].value_counts()
numbers = ratings.index
quantity = ratings.values 

figure = px.pie(data, values = quantity, labels=numbers)
figure.show()

In [8]:
data['Rating'].unique()

array([5, 4, 3, 2, 1])

In [10]:
pivot = pd.pivot_table(data, values ='Product_name', index = 'Rating', aggfunc = 'count' )
pivot

,Product_name
Rating,
1,184
2,46
3,140
4,552
5,1382
